# Basic usage of holographic force measurement software

## Initialization

In [ ]:
%pylab inline

In [ ]:
import holoforce

In [ ]:
holoforce.__version__

### Configure GPU

`holoforce` uses GPU acceleration via OpenCL. It requires the packages `pyopencl` and [gpyfft](https://github.com/geggo/gpyfft)

Select GPU device

In [ ]:
import pyopencl as cl
cl_platform = cl.get_platforms()[0]  # use first platform
cl_device = cl_platform.get_devices(cl.device_type.GPU)[0]  # use first GPU found

cl_device

In [ ]:
cl_context = cl.Context([cl_device])
cl_queue = cl.CommandQueue(cl_context)

from pyopencl.tools import MemoryPool, ImmediateAllocator
cl_allocator = MemoryPool(ImmediateAllocator(cl_queue))

## Load measurement data

Data from measurement with 3 µm silica microspheres.

In [ ]:
with load('data/Data_single_image_with_empty.npz') as d:
    # images intermediate focal plane for calibration of aperture placement
    img_mask = d['img_mask']
    img_spots = d['img_spots']
    spot_positions = d['spot_positions']

    # measured ingoing intensity distribution on SLM
    I0 = d['I0']

    # background image for back focal plane
    bg_bfp = d['bg_bfp'] 

    #positions of traps in object plane
    trap_positions = d['trap_positions']
    # measured back focal plane intensity
    bfp = d['bfp']
    # measured back focal plane intensity for empty trap
    bfp_empty = d['bfp_empty'] 

    #phase pattern on slm for choosen trap configuration
    phase_slm = d['phase_slm'] 
    #phase pattern containing only aberration correction pattern
    phase_slm0 = d['phase_slm0'] 

## Measurement of aperture placement in intermediate focal plane

Use images from auxiliary camera of the intermediate focal plane to find position of aperture. Requires two images:
* image of four spots to determine optical axis, scaling and rotation
* image with diffuser pattern (random phase) to completely illuminate focal plane


In [ ]:
import holoforce.create_mask

In [ ]:
M = holoforce.create_mask.CreateMaskFocal(img_mask = img_mask, img_spots = img_spots, pos_spots_holo = spot_positions, field_shape = (1024,) * 2)
M.create_mask(sigma = 0.3, gamma = .6, dpi = 80, figsize = (12,3)) #sigma is most important to change if not all pixels are detected!

In [ ]:
fourier_mask = M.mask #.copy()
fourier_plane_size = M.field_size
fourier_plane_shape = M.field_shape
print('Fourier plane size = %.3f mm' %(fourier_plane_size[0]*1e3))

## Preprocess illumination pattern

In [ ]:
# function to create circular mask
def create_circular_mask(shape, center=(0,0), radius=1.02):
    
    x = linspace(-1,1,shape[1])
    y = linspace(-1,1,shape[0])
                   
    X, Y = np.meshgrid(x,y, sparse=True)
    R2 = np.square(X - center[0]) + np.square(Y - center[1])
    mask = R2 <= radius**2
    return mask#.astype(float)

In [ ]:
#apply a mask to the intensity distribution I0 in order to discard stray light
mask_I0 = create_circular_mask(I0.shape, radius=0.915, center=(0.001, 0.00))
I0_masked = clip ( (I0 - bg_bfp)*mask_I0, 0 , None)

# compensate for reduced diffraction efficiency for calibration data I0
scale_I0 = 1.11
I0_masked *= scale_I0

figure()
imshow( I0_masked, vmin=0.01, 
       #cmap=palette
      )
axis('off')

## Preprocess back focal plane data

In [ ]:
# multiply measured bfp with mask to discard stray light
mask_bfp = create_circular_mask(bfp.shape)
bfp_measured = ((bfp-bg_bfp)*mask_bfp).astype(float32)
bfp_measured_empty = ((bfp_empty-bg_bfp)*mask_bfp).astype(float32)

## Fieldretrieval for a single BFP image

Select active trap indices

In [ ]:
# active traps
active_trap_idx = [0,1]  # indices of active traps
trap_pos = trap_positions[active_trap_idx, :] * 1e-6 #in um

### Retrieve field

In [ ]:
# Initialization

PSF_detector = holoforce.fieldretriever.double_gaussian_kernel(
    N=1024, sigma_1=.9, sigma_2=40., p2=0.12).astype(float32)

R = holoforce.fieldretriever.FieldRetrieverGPU(cl_context=cl_context, cl_queue=cl_queue, cl_allocator=cl_allocator,
              slm_phase = phase_slm,
              slm_phase0 = phase_slm0,
              I0 = I0_masked,
              R0 = 1.02,
              fourier_plane_size = fourier_plane_size,
              fourier_plane_shape = fourier_plane_shape,
              object_plane_shape = fourier_plane_shape,
              fourier_plane_mask = fourier_mask,
              detector_psf = PSF_detector)

R.init_all(pos=trap_pos, patch_size = 5e-6)

In [ ]:
imshow(log10(R.object_multiareafield.field.intensity),vmin = 0.1, vmax = 8, cmap = cm.gray_r)
title('reference field in object plane\nlogarithmic colormap')

### Check for energy conservation

Check that total power in measured back focal plane (`bfp_measured`) equals ingoing power, otherwise adjust `scale_I0` above.

In [ ]:
#calculate initial back focal plane intensity
retrieved_init = R.retrieve_field(bfp_measured, iterations = 0)[0]

print('ratio = %.2f'%(bfp_measured.sum()/retrieved_init.sum()))

In [ ]:
#iterative field retrieval
retrieved, retrieved_no_transmission = R.retrieve_field(bfp_measured, iterations = 50, stepsize = 500, momentum = 0.85)

In [ ]:
#plot loss function vs. number of iterations
plot(R.log)
xlabel('iteration')
ylabel('residuum')

In [ ]:
#compare measured and retrieved BFP data

retrieved = retrieved
original = bfp_measured

v_min = 0
v_max = max(original.max(), retrieved.max())
v_max_residuum = 0.2 * v_max

r, c = slice(None, None), slice(None, None)
#r, c = slice(400,600), slice(400,600)

fig, (a1,a2,a3) = subplots(1,3, figsize=(12,4))
a1.imshow(original[r,c], vmin = v_min, vmax = v_max)
a2.imshow(retrieved[r,c], vmin = v_min, vmax = v_max)
a3.imshow((original - retrieved)[r,c], vmin = -v_max_residuum, vmax = v_max_residuum, cmap = cm.RdBu_r)
a1.set_title('measured bfp')
a2.set_title('retrieved bfp')
a3.set_title('difference')
fig.tight_layout()

In [ ]:
K = 512
s0 = slice(None, K)
s1 = slice(K, None)
s00 = s0, s0
s01 = s0, s1
s10 = s1, s0
s11 = s1, s1

d = original.copy()
d[s01] = retrieved[s01]
xp, yp = [], []
for n in range(512, 1024,16)[::2]:
    s = slice(n, n+16), s0
    d[s] = retrieved[s]
    xp.extend([0, K-.5, NaN, 0, K-.5, NaN])
    yp.extend([n-.5, n-.5, NaN, n+16-.5, n+16-.5, NaN])
    
d[s11] = (original - retrieved)[s11]*-2 + v_max/2

figure()
imshow(d, vmin=0, vmax=0.8*v_max, cmap = plt.cm.RdBu_r, interpolation='lanczos')
plot([0, 1024, NaN, 511.5, 511.5], [511.5, 511.5, NaN, 0, 1024], 'w-', lw=.5, alpha=.5)
plot(xp, yp, 'w-', lw=.5, alpha=.5)
axis('off')

xlim(200, 800)
ylim(700, 300)
title('comparison retrieved and original bfp')

In [ ]:
# propagate indiviudal retrieved patches
R.calculate_individual_farfields(just_intesities = True)
ind_bfps = R.individual_farfield_intensities.copy()

In [ ]:
# individual farfield intensity
v_max = 1.4*R.individual_farfield_intensities[0].max()

fig, ax = subplots(1,len(R.object_subfields_0), sharex=True, sharey=True, squeeze=True, tight_layout = True)
for i, ind_bfp in enumerate(ind_bfps):
    ax[i].imshow(ind_bfp, vmin=0, vmax=0.85*v_max)    
    ax[i].axis('off')
ax[0].set_title('bfp trap 1')
ax[1].set_title('bfp trap 2')

### Retrieval for BFP with empty traps (for ingoing moementum flux F_0)

In [ ]:
retrieved_empty = R.retrieve_field(bfp_measured_empty, iterations = 30, stepsize = 500, momentum = 0.85)[0]

In [ ]:
R.calculate_individual_farfields( just_intesities = True)
ind_bfps_empty = R.individual_farfield_intensities.copy()

In [ ]:
# individual farfield intensity
v_max = 1.4*R.individual_farfield_intensities[0].max()

fig, ax = subplots(1,len(R.object_subfields_0), sharex=True, sharey=True, squeeze=True, tight_layout = True)
for i, ind_bfp in enumerate(ind_bfps_empty):
    ax[i].imshow(ind_bfp, vmin=0, vmax=0.85*v_max)    
    ax[i].axis('off')
ax[0].set_title('empty bfp trap 1')
ax[1].set_title('empty bfp trap 2')

## calculate force from retrieved individual back focal plane data

In [ ]:
import holoforce.force

In [ ]:
C = holoforce.force.CalculateForce(R0 = R.R0, scale_au_to_pN = 1/45000)

In [ ]:
F_ind = array(([C.calc_force(I) for I in ind_bfps]))
F_ind_0 = array(([C.calc_force(I) for I in ind_bfps_empty]))
F_ind_e = F_ind-F_ind_0

In [ ]:
print(r'Forces trap 1 = (%.2f, %.2f, %.2f) pN'%(F_ind_e[0,0],F_ind_e[0,1], F_ind_e[0,2]))
print(r'Forces trap 2 = (%.2f, %.2f, %.2f) pN'%(F_ind_e[1,0],F_ind_e[1,1], F_ind_e[1,2]))